# Purpose

This notebook demonstrates the model experimentation and finalization. It covers EDA, outlier treatment, transformation, training, model evaluation and comparison across models.

## Imports

In [37]:
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

# impute missing values
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder


In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import warnings

warnings.filterwarnings('ignore', message="pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.", 
                        category=FutureWarning)
warnings.filterwarnings('ignore', message="pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.",
                        category=FutureWarning)

In [40]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, string_cleaning,
    get_package_path, display_as_tabs, save_pipeline, load_pipeline, initialize_environment,
    load_dataset, save_dataset, DEFAULT_ARTIFACTS_PATH, list_datasets
)

import ta_lib.eda.api as eda
from xgboost import XGBRegressor
from ta_lib.regression.api import SKLStatsmodelOLS
from ta_lib.regression.api import RegressionComparison, RegressionReport
import ta_lib.reports.api as reports
from ta_lib.data_processing.api import Outlier

initialize_environment(debug=False, hide_warnings=True)

# Initialization

In [41]:
artifacts_folder = DEFAULT_ARTIFACTS_PATH

In [42]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [43]:
list_datasets(context)

['/raw/orders',
 '/raw/product',
 '/cleaned/orders',
 '/cleaned/product',
 '/cleaned/sales',
 '/processed/google_search',
 '/processed/sales',
 '/processed/social_media',
 '/processed/ground_truth',
 '/train/sales/features',
 '/train/sales/target',
 '/test/sales/features',
 '/test/sales/target',
 '/score/sales/output']

In [45]:
google_search_data = load_dataset(context,  '/processed/google_search')
sales_data = load_dataset(context, '/processed/sales')
social_media_data = load_dataset(context, '/processed/social_media')

# 2 EDA/Transformations

In [9]:
# TODO : Modularize the whole code

In [46]:
sales_ground_truth = sales_data.copy()
social_media_ground_truth = social_media_data.copy()
google_search_data_ground_truth = google_search_data.copy()

In [47]:
def filter_out_common_themes(dataframe: pd.DataFrame) -> tuple:
    """
    Filters out the common themes.
    
    Arguments
    ---------
    dataframe : pd.DataFrame
        The dataframe to filter out the themes from.
    
    Returns
    -------
    tuple
        A tuple of dataframes.
    """
    common_theme_list = set(sales_ground_truth['theme_name']) & set(social_media_ground_truth['theme_name']) & set(google_search_data_ground_truth['theme_name'])
    
    temp_dataframe = dataframe[dataframe['theme_name'].isin(common_theme_list)]
    
    return temp_dataframe
    

In [50]:
sales_ground_truth = filter_out_common_themes(sales_ground_truth)
social_media_ground_truth = filter_out_common_themes(social_media_ground_truth)
google_search_data_ground_truth = filter_out_common_themes(google_search_data_ground_truth)

In [51]:
sales_ground_truth['theme_name'].nunique()

30

In [52]:
def get_top_3_themes_by_sales_in_lbs(dataframe: pd.DataFrame) -> list:
    """
    Fetches the top 3 sales by lbs.
    
    Arguments
    ---------
    dataframe
        The dataframe to use for getting the top 3 sales.
        
    Returns
    -------
    list
        A list of the top 3 themes.
    """
    
    temp_dataframe = dataframe.copy()

    total_sales_data = temp_dataframe[['date', 'theme_name', 'sales_lbs_value']]

    total_sales_data = total_sales_data[total_sales_data['theme_name'] != 'No Claim']
    total_sales_data['total_sales_by_theme'] = total_sales_data.groupby(['theme_name'])['sales_lbs_value'].transform('sum')

    total_sales_data = total_sales_data[['theme_name', 'total_sales_by_theme']]
    total_sales_data.drop_duplicates(inplace=True)

    # We are taking the top 3 themes by sales in lbs
    total_sales_data.sort_values(by=['total_sales_by_theme'], ascending=False, inplace=True)

    total_sales_data = total_sales_data[:3]

    top_themes = total_sales_data['theme_name'].unique().tolist()
    
    return top_themes

In [53]:
top_themes = get_top_3_themes_by_sales_in_lbs(sales_ground_truth)

In [54]:
print(top_themes)

['low carb', 'no additives/preservatives', 'salmon']


In [55]:
def filter_out_top_three_themes(dataframe_list : pd.DataFrame, theme_list: list) -> None:
    """
    Filter out the top 3 themes from the dataframes provided.
    
    Arguments
    ---------
    dataframe_list : list[pd.DataFrame]
        List of dataframes to apply the filtering on.
        
    theme_list : list
        The list of themes to use for filtering.
        
    """
    
    for dataframe in dataframe_list:
        dataframe = dataframe[dataframe['theme_name'].isin(theme_list)]
    print('Dataframes filtered successfully!')

In [56]:
filter_out_top_three_themes([sales_ground_truth, social_media_ground_truth, google_search_data_ground_truth], top_themes)

Dataframes filtered successfully!


In [57]:
social_media_ground_truth['theme_name'].nunique()

30

In [58]:
# Taking only the top 3 themes
sales_ground_truth = sales_ground_truth[sales_ground_truth['theme_name'].isin(top_themes)]
social_media_ground_truth = social_media_ground_truth[social_media_ground_truth['theme_name'].isin(top_themes)]
google_search_data_ground_truth = google_search_data_ground_truth[google_search_data_ground_truth['theme_name'].isin(top_themes)]

In [59]:
google_search_data_ground_truth.shape

(632, 9)

In [60]:
def aggregate_google_search_data(google_search_ground_truth: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates the google search data.
    
    Arguments
    ---------
    google_search_ground_truth : pd.DataFrame
        The google search dataframe to aggregate.
        
    Returns
    -------
    pd.DataFrame
        The aggregated dataframe.
    """
    
    temp_dataframe = google_search_data.copy()
    required_cols = ['date','theme_name'
     , 'google_searchVolume', 'amazon_searchVolume', 'chewy_searchVolume', 'walmart_searchVolume', 'total_searchVolume']
    temp_dataframe = temp_dataframe.filter(required_cols)


    temp_dataframe.drop_duplicates(inplace=True)
    
    return temp_dataframe
    

In [61]:
google_search_data_ground_truth = aggregate_google_search_data(google_search_data_ground_truth)

In [62]:
google_search_data_ground_truth.shape

(28600, 7)

In [63]:
def aggregate_social_media_data(social_media_ground_truth: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates the google search data.
    
    Arguments
    ---------
    social_media_ground_truth : pd.DataFrame
        The social media data to aggregate.
        
    Returns
    -------
    pd.DataFrame
        The aggregated dataframe.
    """
    
    temp_dataframe = social_media_ground_truth.copy()
    required_cols = ['date', 'theme_name', 'total_post']

    temp_dataframe = temp_dataframe.filter(required_cols)


    temp_dataframe.drop_duplicates(inplace=True)
    
    return temp_dataframe
    

In [64]:
social_media_ground_truth = aggregate_social_media_data(social_media_ground_truth)

In [65]:
social_media_ground_truth.sample(10)

,date,theme_name,total_post
196,2018-12-15,low carb,2531
187,2018-10-13,low carb,2096
14,2015-06-20,low carb,913
31,2015-10-17,low carb,1506
9555,2018-10-27,salmon,1935
9553,2018-10-13,salmon,603
1288,2016-03-26,no additives/preservatives,37
9583,2019-05-11,salmon,981
1454,2019-06-01,no additives/preservatives,226
27,2015-09-19,low carb,1809


In [66]:
def aggregate_sales_data(sales_ground_truth: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate the sales data.
    
    Arguments
    ---------
    sales_ground_truth : pd.DataFrame
        The sales data to aggregate.
        
    Returns
    -------
    pd.DataFrame
        The aggregated sales data.
    """
    temp_dataframe = sales_ground_truth.copy()
    temp_dataframe['date'] = pd.to_datetime(temp_dataframe['date'])


    required_cols = ['date', 'sales_dollars_value', 'sales_units_value', 'sales_lbs_value', 'vendor', 'theme_name']
    temp_dataframe = temp_dataframe.filter(required_cols)

    temp_dataframe['weekly_lbs_value'] = temp_dataframe.groupby(['date', 'theme_name', 'vendor'])['sales_lbs_value'].transform('sum')
    temp_dataframe['weekly_units_value'] = temp_dataframe.groupby(['date', 'theme_name', 'vendor'])['sales_units_value'].transform('sum')
    temp_dataframe['weekly_dollars_value'] = temp_dataframe.groupby(['date', 'theme_name', 'vendor'])['sales_dollars_value'].transform('sum')
    temp_dataframe = temp_dataframe[['date', 'theme_name', 'vendor', 'weekly_lbs_value', 'weekly_units_value', 'weekly_dollars_value']]
    temp_dataframe.drop_duplicates(inplace=True)
    
    return temp_dataframe
    
    

In [67]:
sales_ground_truth = aggregate_sales_data(sales_ground_truth)

In [68]:
def convert_date_columns_to_datetime():
    sales_ground_truth['date'] = pd.to_datetime(sales_ground_truth['date'])
    social_media_ground_truth['date'] = pd.to_datetime(social_media_ground_truth['date'])
    google_search_data_ground_truth['date'] = pd.to_datetime(google_search_data_ground_truth['date'])
    
    return sales_ground_truth, social_media_ground_truth, google_search_data_ground_truth

In [69]:
sales_ground_truth, social_media_ground_truth, google_search_data_ground_truth = convert_date_columns_to_datetime()

In [70]:
def create_final_merged_data(sales_ground_truth: pd.DataFrame, social_media_ground_truth: pd.DataFrame, google_search_ground_truth : pd.DataFrame) -> pd.DataFrame:
    """
    Creates the final ground truth to use further downstream.
    
    Arguments
    ---------
    sales_ground_truth : pd.DataFrame
        The sales ground truth to use for the final merging process.
    social_media_ground_truth : pd.DataFrame
        The social ground truth data to use for the final merging process.
    google_search_ground_truth : pd.DataFrame
        The google search ground truth data to use for the final merging process.
    
    Returns
    -------
    pd.DataFrame
        The merged dataframe
    """
    combined_ground_truth = pd.merge(sales_ground_truth, social_media_ground_truth, on=['date', 'theme_name'])
    combined_ground_truth = pd.merge(combined_ground_truth, google_search_ground_truth, on=['date', 'theme_name'])
    return combined_ground_truth

In [71]:
combined_ground_truth = create_final_merged_data(sales_ground_truth, social_media_ground_truth, google_search_data_ground_truth)

In [72]:
combined_ground_truth.shape

(2940, 12)

In [73]:
def process_vendor_wise_data(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Process the vendor wise data.
    
    Arguments
    ---------
    dataframe : pd.DataFrame
        The dataframe containing the vendor wise data.
        
    Returns
    -------
    pd.DataFrame
        The processed dataframe containing client and competitor related features.
    """
    
    # Client related features
    client_dataframe = dataframe[dataframe['vendor'] == 'A']
    client_dataframe.rename(columns={'weekly_lbs_value': 'client_lbs_value'
                                     , 'weekly_units_value': 'client_units_value'
                                     , 'weekly_dollars_value': 'client_dollars_value'}, inplace=True)
    
    # Competitor related features
    competitor_dataframe = dataframe[dataframe['vendor'] != 'A']
    competitor_dataframe['competitor_lbs_value'] = competitor_dataframe.groupby(['date', 'theme_name'])['weekly_lbs_value'].transform('sum')
    competitor_dataframe['competitor_units_value'] = competitor_dataframe.groupby(['date', 'theme_name'])['weekly_units_value'].transform('sum')
    competitor_dataframe['competitor_dollars_value'] = competitor_dataframe.groupby(['date', 'theme_name'])['weekly_dollars_value'].transform('sum')
    
    client_dataframe = client_dataframe[['date', 'theme_name', 'client_lbs_value', 'client_units_value', 'client_dollars_value']]
    competitor_dataframe = competitor_dataframe[['date', 'theme_name', 'competitor_lbs_value', 'competitor_units_value', 'competitor_dollars_value']]
    
    dataframe = pd.merge(dataframe, client_dataframe, on=['date', 'theme_name'])
    dataframe = pd.merge(dataframe, competitor_dataframe, on=['date', 'theme_name'])
    
    dataframe = dataframe[['date', 'theme_name', 'total_post'
                                                    , 'google_searchVolume', 'amazon_searchVolume'
                                                    , 'chewy_searchVolume', 'walmart_searchVolume'
                                                    , 'total_searchVolume', 'client_lbs_value',
                                                    'client_units_value', 'client_dollars_value',
                                                    'competitor_lbs_value', 'competitor_units_value',
                                                    'competitor_dollars_value'
                                                   ]]
    dataframe.drop_duplicates(inplace=True)
    return dataframe

In [74]:
combined_ground_truth = process_vendor_wise_data(combined_ground_truth)

In [75]:
combined_ground_truth

,date,theme_name,total_post,google_searchVolume,amazon_searchVolume,chewy_searchVolume,walmart_searchVolume,total_searchVolume,client_lbs_value,client_units_value,client_dollars_value,competitor_lbs_value,competitor_units_value,competitor_dollars_value
0,2016-01-09,salmon,47,1913.0,0.0,0.0,0.0,1913.0,5.379338e+06,754859,7.639690e+06,4.554449e+06,657551,4.899568e+06
42,2016-02-06,salmon,47,850.0,0.0,0.0,0.0,850.0,5.217845e+06,745221,7.433237e+06,5.106687e+06,645078,4.990354e+06
84,2016-02-20,salmon,156,1063.0,0.0,0.0,0.0,1063.0,4.747996e+06,687313,6.826838e+06,4.962688e+06,669369,4.958131e+06
126,2016-02-27,salmon,90,1912.0,0.0,0.0,0.0,1912.0,4.676554e+06,680633,6.724496e+06,4.954023e+06,667029,4.956457e+06
168,2016-03-26,salmon,50,1912.0,0.0,0.0,0.0,1912.0,4.739258e+06,705062,6.837560e+06,4.807323e+06,640089,4.817914e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17430,2019-09-07,low carb,1891,38008.0,5236.0,4489.0,627.0,48360.0,4.028791e+06,2695393,1.331764e+07,2.992734e+07,12652417,8.959849e+07
17472,2019-09-14,low carb,1142,41306.0,4655.0,4862.0,669.0,51492.0,4.017835e+06,2682848,1.327477e+07,2.982720e+07,12612445,8.869762e+07
17514,2019-09-21,low carb,1158,35338.0,3699.0,2952.0,625.0,42614.0,3.974981e+06,2641925,1.313288e+07,2.985235e+07,12537084,8.910157e+07
17556,2019-09-28,low carb,1545,33610.0,6315.0,2661.0,960.0,43546.0,3.980566e+06,2684050,1.301893e+07,2.997781e+07,12565509,8.939163e+07


In [76]:
ground_truth_for_modelling = combined_ground_truth[['date', 'theme_name', 'total_post'
                                                    , 'google_searchVolume', 'amazon_searchVolume'
                                                    , 'chewy_searchVolume', 'walmart_searchVolume'
                                                    , 'total_searchVolume', 'client_lbs_value',
                                                    'client_units_value', 'client_dollars_value',
                                                    'competitor_lbs_value', 'competitor_units_value',
                                                    'competitor_dollars_value'
                                                   ]]
ground_truth_for_modelling.drop_duplicates(inplace=True)

In [77]:
save_dataset(context, ground_truth_for_modelling, '/processed/ground_truth')

In [78]:
features = X = ground_truth_for_modelling.drop(columns=['client_lbs_value'])
target = y = ground_truth_for_modelling['client_lbs_value']

In [79]:
list_datasets(context)

['/raw/orders',
 '/raw/product',
 '/cleaned/orders',
 '/cleaned/product',
 '/cleaned/sales',
 '/processed/google_search',
 '/processed/sales',
 '/processed/social_media',
 '/processed/ground_truth',
 '/train/sales/features',
 '/train/sales/target',
 '/test/sales/features',
 '/test/sales/target',
 '/score/sales/output']

In [ ]:
# TODO : Move this to 2nd notebook

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [81]:
save_dataset(context, X_train, '/train/sales/features')
save_dataset(context, y_train, '/train/sales/target')

In [82]:
save_dataset(context, X_test, '/test/sales/features')
save_dataset(context, y_test, '/test/sales/target')

# 3 Feature Engineering

The focus here is the `Pipeline` and not the model. Though the model would inform the pipeline that is needed to train the model, our focus is to set it up in such a way that it can be saved/loaded, tweaked for different model choices and so on.

## 3.1 Read the Train and Test Data

In [ ]:
# TODO: Check why the shape is having a mismatch 

In [83]:
train_X = load_dataset(context, 'train/sales/features')
train_y = load_dataset(context, 'train/sales/target')
print(train_X.shape, train_y.shape)

test_X = load_dataset(context, 'test/sales/features')
test_y = load_dataset(context, 'test/sales/target')
print(test_X.shape, test_y.shape)

(294, 13) (294, 1)
(126, 13) (126, 1)


## 3.2 Feature Engineering Pipelines


**Dev NOTES**

For Feature Engineering and Model Building sklearn.pipeline.Pipeline are leveraged because of the following advantages
<details>
    
1. It helps in automating workflows and are easier to read and comprehend.
2. Right Sequence can be ensured and (for example always encodes before imputing)
3. Reproducibility is very convenient with pipelines
4. Pipelines help you prevent data leakage in your test data
5. Code is near implementation ready

#### General Steps in the Feature Transformation are as follows
 - Outlier Treatment
 - Encoding of Categorical Columns
 - Missing Values Imputation

In [84]:
# collecting different types of columns for transformations
cat_columns = train_X.select_dtypes('object').columns
num_columns = train_X.select_dtypes('number').columns

In [85]:
cat_columns

Index(['theme_name'], dtype='object')

In [86]:
num_columns

Index(['total_post', 'google_searchVolume', 'amazon_searchVolume',
       'chewy_searchVolume', 'walmart_searchVolume', 'total_searchVolume',
       'client_units_value', 'client_dollars_value', 'competitor_lbs_value',
       'competitor_units_value', 'competitor_dollars_value'],
      dtype='object')

#### Outlier Handling
- A Custom Transformer is used to handle outliers. It is not included as part of the pipeline as outliers handling are optional for test data
- An option to either drop or cap the outliers can be passed during the transform call
- If we want to treat outliers for some columns them we can pass cols argument to the Transformer
- This will go into production code

In [87]:
outlier_transformer = Outlier(method='median')
print(train_X.shape)
train_X = outlier_transformer.fit_transform(train_X)
print(train_X.shape)

(294, 13)
(294, 13)


#### Encoding


Some sample pipelines showcasing how to create column specific pipelines and integrating them overall is presented below

- Commonly target encoding is done for categorical variables with too many levels.
- We also group sparse levels. For fewer levels one hot encoding/label encoding is preferred.
- If there is one dominant level, we can use binary encoding.
- This will go into production code

In [88]:
# TODO : Remove mode imputer
cat_imputer = Pipeline([
    ('simple_impute', SimpleImputer(strategy='most_frequent')),
])


# NOTE: the list of transformations here are not sequential but weighted 
# (if multiple transforms are specified for a particular column)
# for sequential transforms use a pipeline as shown above.
features_transformer = ColumnTransformer([
    
    ## categorical columns
    ('one_hot_enc', OneHotEncoder(drop='first'),
     list(set(cat_columns))),
    
    # NOTE: if the same column gets repeated, then they are weighed in the final output
    # If we want a sequence of operations, then we use a pipeline but that doesen't YET support
    # get_feature_names. 
    ('cat_variable_imputer', cat_imputer, ['theme_name']),
        
    ## numeric columns
    ('med_enc', SimpleImputer(strategy='median'), num_columns),
    
])


**Dev notes(Encoding):**
<details>

    Some common practices followed in Categorical Feature Encoding are
    * For categorical variables with too many levels, target encoding can be done.
    * For fewer levels, one hot encoding can be done.
    * If one very dominant level is observed, binary encoding can be used.
    
    
</details>

## 3.2 Feature analysis

Using the pipeline above analyze the features and decide on additional features to add/remove from the pipeline. This section will not be part of the production code, unless input data drifts etc. are explicitly demanded in the project.

Here we are primarily focused on feature selection/elimination based on business rules, prior knowledge, data analysis.

**We are not building any models at this point.**


- we create some sample data to analyze that we assume represent the population
- train the features transformer and do the analysis as below

In [89]:
sample_X = train_X.sample(frac=0.1, random_state=context.random_seed)
sample_y = train_y.loc[sample_X.index]

sample_train_X = get_dataframe(
    features_transformer.fit_transform(sample_X, sample_y), 
    get_feature_names_from_column_transformer(features_transformer)
)

# nothing to do for target
sample_train_y = sample_y

Running the features transformer on the complete data

In [90]:
train_X = get_dataframe(
    features_transformer.fit_transform(train_X, train_y), 
    get_feature_names_from_column_transformer(features_transformer)
)

In [ ]:
# TODO: Check why the additional column of cat_variable_imputer_theme_name is being created

In [91]:
train_X

,theme_name_no additives/preservatives,theme_name_salmon,cat_variable_imputer_theme_name,total_post,google_searchVolume,amazon_searchVolume,chewy_searchVolume,walmart_searchVolume,total_searchVolume,client_units_value,client_dollars_value,competitor_lbs_value,competitor_units_value,competitor_dollars_value
0,0.0,1.0,salmon,113.0,1700.0,0.0,0.0,0.0,1700.0,728731.0,6867987.0,4499792.0,653720.0,4420861.0
1,0.0,1.0,salmon,2649.0,12052.0,532.0,0.0,0.0,12584.0,719860.0,7872415.0,4917567.0,784836.0,4494954.899164
2,0.0,0.0,low carb,1781.0,39684.0,1345.0,101.25,0.0,49955.0,2631979.0,13325897.0,32110762.680918,12581999.0,92941796.298751
3,0.0,1.0,salmon,259.0,3159.0,94.0,0.0,0.0,3253.0,647701.0,7017078.0,4760472.294024,788253.0,4473418.087173
4,0.0,1.0,salmon,360.0,4323.0,0.0,0.0,0.0,4323.0,686618.0,7300266.0,4855674.562517,793087.0,4405833.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0.0,1.0,salmon,500.0,7713.0,0.0,0.0,0.0,7713.0,700644.0,7334551.0,4189191.564075,731141.0,4386358.0
290,0.0,1.0,salmon,4630.875,3542.0,178.0,0.0,0.0,3720.0,664036.0,7172574.0,4860118.271304,787082.0,4476651.0
291,0.0,0.0,low carb,2378.0,40544.0,0.0,0.0,0.0,40544.0,2585867.0,11181066.0,26064293.944524,9212192.0,73580900.766766
292,0.0,0.0,low carb,3283.0,38589.0,1345.0,101.25,0.0,55397.0,2792590.0,13377807.148767,30890438.81955,11165841.0,86136270.603325


### 3.2.1 Univariate


- Look at each variable independently. This is useful if your models have assumptions on the distribution and/or bounds on the features/target

In [92]:
train_X.columns

Index(['theme_name_no additives/preservatives', 'theme_name_salmon',
       'cat_variable_imputer_theme_name', 'total_post', 'google_searchVolume',
       'amazon_searchVolume', 'chewy_searchVolume', 'walmart_searchVolume',
       'total_searchVolume', 'client_units_value', 'client_dollars_value',
       'competitor_lbs_value', 'competitor_units_value',
       'competitor_dollars_value'],
      dtype='object')

In [93]:
# TODO: Remove this , temporary code
train_X = train_X.drop(columns=['cat_variable_imputer_theme_name'])

In [94]:
train_X.columns

Index(['theme_name_no additives/preservatives', 'theme_name_salmon',
       'total_post', 'google_searchVolume', 'amazon_searchVolume',
       'chewy_searchVolume', 'walmart_searchVolume', 'total_searchVolume',
       'client_units_value', 'client_dollars_value', 'competitor_lbs_value',
       'competitor_units_value', 'competitor_dollars_value'],
      dtype='object')

In [95]:
train_X['theme_name_no additives/preservatives'] = train_X['theme_name_no additives/preservatives'].astype(int)
train_X['theme_name_salmon'] = train_X['theme_name_salmon'].astype(int)

In [96]:
for column in num_columns:
    train_X[column] = train_X[column].astype(float)

In [97]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   theme_name_no additives/preservatives  294 non-null    int64  
 1   theme_name_salmon                      294 non-null    int64  
 2   total_post                             294 non-null    float64
 3   google_searchVolume                    294 non-null    float64
 4   amazon_searchVolume                    294 non-null    float64
 5   chewy_searchVolume                     294 non-null    float64
 6   walmart_searchVolume                   294 non-null    float64
 7   total_searchVolume                     294 non-null    float64
 8   client_units_value                     294 non-null    float64
 9   client_dollars_value                   294 non-null    float64
 10  competitor_lbs_value                   294 non-null    float64
 11  compet

In [ ]:
out = eda.get_density_plots(train_X, cols=train_X.columns.tolist())
out

In [ ]:
# save the plots are html
reports.create_report({'univariate': out}, name='feature_analysis_univariate')

In [ ]:
reports.feature_analysis(train_X,'./feature_analysis_report.html')

### 3.2.2 Bivariate - mutual interactions

- Find columns with high correlations and drop them

In [98]:
out = eda.get_correlation_table(train_X)
out[out["Abs Corr Coef"] > 0.6]

,Variable 1,Variable 2,Corr Coef,Abs Corr Coef
0,competitor_dollars_value,competitor_lbs_value,0.998105,0.998105
1,competitor_dollars_value,competitor_units_value,0.993704,0.993704
2,competitor_lbs_value,competitor_units_value,0.987295,0.987295
3,competitor_lbs_value,theme_name_salmon,-0.981824,0.981824
4,google_searchVolume,total_searchVolume,0.980723,0.980723
5,competitor_dollars_value,theme_name_salmon,-0.979964,0.979964
6,client_units_value,competitor_units_value,0.974897,0.974897
7,client_units_value,competitor_dollars_value,0.962097,0.962097
8,competitor_units_value,theme_name_salmon,-0.956849,0.956849
9,client_units_value,competitor_lbs_value,0.955041,0.955041


In [99]:
# Dropping total search volume as its highly correlated with google search volume
# Dropping chewy search volume as its highly correlated with amazon search volume
# We will be going forward with the rest of the features as they represent key 
# features to account for after the modelling stage(despite the high correlations).
curated_columns = list(
    set(train_X.columns.to_list()) 
    - set(['total_searchVolume', 'chewy_searchVolume'])
)

train_X = train_X[curated_columns]

out = eda.get_correlation_table(train_X)
out[out["Abs Corr Coef"] > 0.6]

,Variable 1,Variable 2,Corr Coef,Abs Corr Coef
0,competitor_dollars_value,competitor_lbs_value,0.998105,0.998105
1,competitor_dollars_value,competitor_units_value,0.993704,0.993704
2,competitor_lbs_value,competitor_units_value,0.987295,0.987295
3,competitor_lbs_value,theme_name_salmon,-0.981824,0.981824
4,competitor_dollars_value,theme_name_salmon,-0.979964,0.979964
5,client_units_value,competitor_units_value,0.974897,0.974897
6,client_units_value,competitor_dollars_value,0.962097,0.962097
7,competitor_units_value,theme_name_salmon,-0.956849,0.956849
8,client_units_value,competitor_lbs_value,0.955041,0.955041
9,client_dollars_value,client_units_value,0.945492,0.945492


In [ ]:
out = eda.get_bivariate_plots(train_X, x_cols=['theme_name_salmon'], y_cols=['theme_name_no additives/preservatives'])
out

In [ ]:
%%time
# create reports as needed
cols = train_X.columns.to_list()
all_plots = {}
for ii, col1 in enumerate(cols): 
    for jj in range(ii+1, len(cols)):
        col2 = cols[jj]
        out = eda.get_bivariate_plots(train_X, x_cols=[col1], y_cols=[col2])
        all_plots.update({f'{col2} vs {col1}': out})

reports.create_report(all_plots, name='feature_analysis_bivariate')

In [ ]:
reports.feature_interactions(train_X,'./feature_interaction_report.html')

### 3.2.3 Key Drivers - Interaction with Target variable

In [ ]:
out = eda.get_target_correlation(train_X, train_y, y_continuous=True)
display_as_tabs([(k, v) for k,v in out.items()])

In [ ]:
train_y['client_lbs_value']

In [ ]:
out = eda.get_feature_importances(train_X, train_y, y_continuous=True)
display_as_tabs([(k, v) for k,v in out.items()])

Key drivers report like feature importance, bivariate plots can be obtained as below

In [ ]:
reports.key_drivers(train_X,train_y, './key_drivers_report.html', y_continuous=True)

**Dev Notes**
<details>
    
- The SHAP plots and bivariate plots in key drivers reports can be obtained by including quick=False as a parameter to key_drivers function call. 
- SHAP plots and bivariate plots often take long depending on data shape.
- The plot with shap is present [here](https://drive.google.com/file/d/1JOTMBLiv3LEqZ-kxZz0RokW9v5UyiGva/view?usp=sharing)

</details>


All the plots like feature analysis, interaction, key drivers can be obtained as a single plot using data exploration method as shown below. The output from this is available [here](https://drive.google.com/file/d/1209MzmSSEhiTYuPfHpaVXFXUVbkaJm0B/view?usp=sharing)

In [ ]:
reports.data_exploration(train_X,train_y,'./data_exploration_report.html', y_continuous=True)

In [ ]:
# saving the list of relevant columns
save_pipeline(curated_columns, op.abspath(op.join(artifacts_folder, 'curated_columns.joblib')))

# save the feature pipeline
save_pipeline(features_transformer, op.abspath(op.join(artifacts_folder, 'features.joblib')))

# 4 Modelling

## 4.1 Modelling - Linear Regression

### 4.1.1 Feature Selection(Specific to Regression)

- Selecting Features specific to regression
- VIF : measure of the amount of multi-collinearity in a set of multiple regressor variables. 
- On a case to case basis VIF thresholds change. Generally 5 or 10 are acceptable levels.
- Usually on a recursive basis when removing the most collinear variable, there can be shuffle in VIF. 
- Often this section will not be part of the production code.

In [ ]:
# TODO: Make sure client and competitor features are not getting removed

In [103]:
cols = list(train_X.columns)
vif = eda.calc_vif(train_X)
while max(vif.VIF) > 12:
    #removing the largest variable from VIF
    cols.remove(vif[(vif.VIF==vif.VIF.max())].variables.tolist()[0])
    vif = eda.calc_vif(train_X[cols])

invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars


In [104]:
reg_vars = vif.query('VIF < 11').variables
reg_vars = list(reg_vars)

In [105]:
reg_vars

['google_searchVolume',
 'total_post',
 'amazon_searchVolume',
 'theme_name_no additives/preservatives',
 'theme_name_salmon']

### 4.1.2 Data transformations

In [101]:
# Custom Transformations like these can be utilised
def _custom_data_transform(df, cols2keep=None):
    """Transformation to drop some columns in the data
    
    Parameters
    ----------
        df - pd.DataFrame
        cols2keep - columns to keep in the dataframe
    """
    cols2keep = cols2keep or []
    if len(cols2keep):
        return (df
                .select_columns(cols2keep))
    else:
        return df

### 4.1.3 Model training pipeline

- Here we focus on creating a collection of pipelines that can be used for training respective models.
- Each model pipeline will essentially be of the form
```
[
('preprocessing', preprocessing_pipeline),
('feature_selection', feature_selection_pipeline),
('estimator', estimator),
]
```

In [ ]:
# TODO : Add feature engineering section

### 4.1.4 Model Pipeline Build

- This will be part of the production code (training only).

In [106]:
reg_ppln_ols = Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':reg_vars})),
    ('estimator', SKLStatsmodelOLS())
])
reg_ppln_ols.fit(train_X, train_y.values.ravel())


Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7f7fef101160>,
                                     kw_args={'cols2keep': ['google_searchVolume',
                                                            'total_post',
                                                            'amazon_searchVolume',
                                                            'theme_name_no '
                                                            'additives/preservatives',
                                                            'theme_name_salmon']})),
                ('estimator', SKLStatsmodelOLS())])

In [107]:
reg_ppln_ols['estimator'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     231.2
Date:                Fri, 09 Jun 2023   Prob (F-statistic):           1.43e-98
Time:                        16:27:21   Log-Likelihood:                -4203.6
No. Observations:                 294   AIC:                             8419.
Df Residuals:                     288   BIC:                             8441.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
intercept                              3.995e+06   1.28e+05     31.128      0.000    3.74e+06    4.25e+06
google_searchVolume                     -10.6856      2.691     -3.971      0.000     -15.982      -5.390
total_post                              117.1397     25.529      4.589      0.000      66.893     167.386
amazon_searchVolume                     450.9269     51.560      8.746      0.000     349.445     552.409
theme_name_no additives/preservatives -8.455e+05   1.53e+05     -5.542      0.000   -1.15e+06   -5.45e+05
theme_name_salmon                      1.224e+06   1.11e+05     11.007      0.000    1.01e+06    1.44e+06
==============================================================================
Omnibus:                       29.448   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.091
Skew:                           0.760   Prob(JB):                     1.45e-08
Kurtosis:                       3.799   Cond. No.                     3.07e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 4.1.5 Model Evaluation(Linear Model)

This will be part of the production code.

In [108]:
reg_ppln = Pipeline([
    ('', FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':reg_vars})),
    ('Linear Regression', SKLStatsmodelOLS())
])

test_X = get_dataframe(
    features_transformer.transform(test_X), 
    get_feature_names_from_column_transformer(features_transformer)
)
test_X = test_X[curated_columns]

In [109]:
reg_linear_report = RegressionReport(model=reg_ppln, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y, refit=True)
reg_linear_report.get_report(include_shap=False, file_path='regression_linear_model_report')

**Dev Notes**
Use SHAP for variable interpretability.
<details>

    1. Use SHAP=True to generate variable interpretability plots in the report
    2. SHAP is recommended for non parameteric models such as RF, xgboost.
    3. However, SHAP reports are time consuming depending on no.of records and model complexity.
    
A sample of regerssion report with SHAP can be found [here](https://drive.google.com/file/d/18RlQTsT1ze09Cgz-qpb4ha_cvyWbN5F5/view?usp=sharing).
</details>

### 4.1.6 Residual Analysis
- After scoring the model, it is recommended to do a residual analysis to know the distribution of errors
- we took a threshold of 30% above which it is marked as over prediction or underprediction
- This will not be part of the production code.

In [110]:
threshold=0.3
residual_analysis = test_X.copy()
residual_analysis['prediction'] = reg_ppln_ols.predict(test_X)
residual_analysis['actuals'] = test_y.reset_index(drop = True).iloc[:,0].values
residual_analysis['forecast_flag'] = 'good'
residual_analysis.loc[((residual_analysis['prediction'] > (1+threshold) * residual_analysis['actuals'])\
                       & (residual_analysis['actuals']>100)),'forecast_flag'] = 'over predict'
residual_analysis.loc[((residual_analysis['prediction'] < (1-threshold) * residual_analysis['actuals'])\
                       & (residual_analysis['actuals']>100)),'forecast_flag'] = 'under predict'

In [ ]:
residual_analysis.hvplot.kde(y="client_lbs_value",by="forecast_flag", ## Grouping by Predictions
                                width=800, height=400,
                                alpha=0.7,
                                ylabel="density",
                                xlabel="unit_cost",
                                title=f'unit cost(density)',legend='top_right')

- From the above plot we can infer that the higher "over predictions" are happening for unit_cost > 200.
- similarly, the higher "under predictions" are happening for unit_cost is zero.

This can help us tune the model by a separate model for unit_cost > 200


# 4.2 Modelling - XGBoost

## 4.2.1 Model training pipeline

Here we focus on creating a collection of pipelines that can be used for tranining respective models.

Each model pipeline will essentially be of the form
```
[
('preprocessing', preprocessing_pipeline),
('feature_selection', feature_selection_pipeline),
('estimator', estimator),
]
```

### 4.2.2 Model Pipeline Build

In [ ]:
# let's find features for some decent defaults
estimator = XGBRegressor()
xgb_training_pipe_init = Pipeline([
    ('XGBoost', XGBRegressor())
])
xgb_training_pipe_init.fit(train_X, train_y)

### 4.2.3 Model Tuning

In [ ]:
# Understanding the Feature Importance
%matplotlib inline
imp = pd.DataFrame({'importance': xgb_training_pipe_init['XGBoost'].feature_importances_})
imp.index = train_X.columns
imp.sort_values('importance',inplace=True)
imp.plot(kind='barh')

'condition','model_family','days_since_last_purchase','first_time_customer','sales_person', are considered to be important and in grid search

##### Pipeline build based on new importance features

In [ ]:
# let's find features for some decent defaults
imp_features = ['model_family','sku','unit_cost','condition','brand','business_unit']

estimator = XGBRegressor()
xgb_training_pipe2 = Pipeline([
    ('', FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':imp_features})),
    ('XGBoost', XGBRegressor())
])

#### Grid Search of the Estimator

In [ ]:
%%time
parameters = {
   'gamma':[0.03],
   'min_child_weight':[6],
   'learning_rate':[0.1],
   'max_depth':[3],
   'n_estimators':[500], 
}
est = XGBRegressor()
xgb_grid = GridSearchCV(est,
                        parameters,
                        cv = 2,
                        n_jobs = 4,
                        verbose=True)

xgb_grid.fit(train_X, train_y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

#### Pipeline Build using the best estimator

In [ ]:
xgb_pipeline_final = Pipeline([
    ('', FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':imp_features})),
    ('XGBoost', xgb_grid.best_estimator_)
])
xgb_pipeline_final.fit(train_X, train_y)

In [ ]:
reg_tree_report = RegressionReport(model=xgb_pipeline_final, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y)
reg_tree_report.get_report(include_shap=False, file_path='regression_tree_model_report')

The Regression report containing the feature importances are available [here](https://drive.google.com/file/d/1JBfL3uxPcxBfl0amweXBFmLr7CSHFBUO/view?usp=sharing)

# 5 Model Comparison

Now, a comparison report of the  linear (vs) tree -based model  approach can be generated as follows.

This code will not be part of the production code.

In [ ]:
model_pipelines = [reg_ppln, xgb_pipeline_final]
model_comparison_report = RegressionComparison(models=model_pipelines,x=train_X, y=train_y)
metrics = model_comparison_report.get_report(file_path='regression_comparison')

In [ ]:
model_comparison_report.performance_metrics

A report comparing the performance, metrics between Linear model and Tree model are available [here](https://drive.google.com/file/d/1LDibiFap9K4DKME-Y0S0mtI_05lTdaJF/view?usp=sharing)

**Dev NOTES**
<details>

the above metrics are absolute nos and not %ges

In this example we are choosing LM model for pipelining. General criteria for choosing production models is:

- Parametric models (aka whitebox models) such as Linear Regression are easier to explain to non-technical audience.
- Generally these are accepted fast and adoption is quicker.
- If the downstream calls for optimization using these models parametric models are easier to implement.
- When accuracy is primary goal without explainability, the above two takes a backseat